In [1]:
from requests import get
from bs4 import BeautifulSoup
import re

README :

밑 코드에서 path 변수 찾아서 자기가 텍스트파일 저장하고싶은 경로로 바꾸세요

### 네이버 시리즈 에디션 연재중 웹소설 내용 스크래핑하기.

In [4]:
# 장르별 url 뽑기

base = "https://novel.naver.com"
gen_index = {'로맨스': 101,
             '로판': 109,
             '판타지': 102,
             '현판': 110,
             '무협': 103}
links_per_gen = {}

for g, i in gen_index.items():
    url = base + f"/webnovel/genre?genre={i}"
  
    # 페이지네이션 처리
    dom = BeautifulSoup(get(url).text, 'html.parser')
    temp = []
    temp.append(url)
    pages = dom.select('.default_paging a')
    for page in pages:
        temp.append(base + page.attrs['href'])
    links_per_gen[g] = temp
    
links_per_gen

{'로맨스': ['https://novel.naver.com/webnovel/genre?genre=101',
  'https://novel.naver.com/webnovel/genre?genre=101&page=2'],
 '로판': ['https://novel.naver.com/webnovel/genre?genre=109',
  'https://novel.naver.com/webnovel/genre?genre=109&page=2',
  'https://novel.naver.com/webnovel/genre?genre=109&page=3',
  'https://novel.naver.com/webnovel/genre?genre=109&page=4',
  'https://novel.naver.com/webnovel/genre?genre=109&page=5',
  'https://novel.naver.com/webnovel/genre?genre=109&page=6'],
 '판타지': ['https://novel.naver.com/webnovel/genre?genre=102'],
 '현판': ['https://novel.naver.com/webnovel/genre?genre=110'],
 '무협': ['https://novel.naver.com/webnovel/genre?genre=103']}

In [5]:
# 장르별 -> 작품별 url 뽑기
links_per_novel = {}
for g, u in links_per_gen.items():
    temp = []
    for link in u:
        dom = BeautifulSoup(get(link).text, 'html.parser')
        alist = dom.select('.card_list li a')
        for a in alist:
            temp.append(base + a.attrs['href'])
            links_per_novel[g] = temp
            
links_per_novel

{'로맨스': ['https://novel.naver.com/webnovel/list?novelId=1051332',
  'https://novel.naver.com/webnovel/list?novelId=1079996',
  'https://novel.naver.com/webnovel/list?novelId=1105605',
  'https://novel.naver.com/webnovel/list?novelId=1079998',
  'https://novel.naver.com/webnovel/list?novelId=1065746',
  'https://novel.naver.com/webnovel/list?novelId=1062254',
  'https://novel.naver.com/webnovel/list?novelId=1082881',
  'https://novel.naver.com/webnovel/list?novelId=1077042',
  'https://novel.naver.com/webnovel/list?novelId=1108680',
  'https://novel.naver.com/webnovel/list?novelId=1113882',
  'https://novel.naver.com/webnovel/list?novelId=1085895',
  'https://novel.naver.com/webnovel/list?novelId=1082882',
  'https://novel.naver.com/webnovel/list?novelId=1079990',
  'https://novel.naver.com/webnovel/list?novelId=1099141',
  'https://novel.naver.com/webnovel/list?novelId=1105607',
  'https://novel.naver.com/webnovel/list?novelId=1108678',
  'https://novel.naver.com/webnovel/list?novelId=

In [6]:
# 작품별 정보(Id, 장르, 작품명, 총 회차, 링크) 생성

novel_info = []

for g, urls in links_per_novel.items():
    for url in urls:
        novel = {}
        dom = BeautifulSoup(get(url).text, 'html.parser')
        
        # id 추가
        novel['id'] = re.search(r'\d{6,}', url)[0]
        # 제목 추가
        novel['title'] = dom.select_one('title').text
        # 작가 추가
        novel['writer'] = dom.select_one(".info_area a").text
        # genre 추가
        novel['genre'] = g
        # 전체 회차 수 추가
        novel['total_epi'] = re.search('\d+', dom.select_one('.past_number').text)[0]

        novel_info.append(novel)

In [7]:
# 위 novel_info에 작품 id별 장르 정보를 넣었기 때문에, 이제부터는 url을 통합관리하도록 하겠음

title_url = []
for k, v in links_per_novel.items():
    title_url.extend(v)
title_url

['https://novel.naver.com/webnovel/list?novelId=1051332',
 'https://novel.naver.com/webnovel/list?novelId=1079996',
 'https://novel.naver.com/webnovel/list?novelId=1105605',
 'https://novel.naver.com/webnovel/list?novelId=1079998',
 'https://novel.naver.com/webnovel/list?novelId=1065746',
 'https://novel.naver.com/webnovel/list?novelId=1062254',
 'https://novel.naver.com/webnovel/list?novelId=1082881',
 'https://novel.naver.com/webnovel/list?novelId=1077042',
 'https://novel.naver.com/webnovel/list?novelId=1108680',
 'https://novel.naver.com/webnovel/list?novelId=1113882',
 'https://novel.naver.com/webnovel/list?novelId=1085895',
 'https://novel.naver.com/webnovel/list?novelId=1082882',
 'https://novel.naver.com/webnovel/list?novelId=1079990',
 'https://novel.naver.com/webnovel/list?novelId=1099141',
 'https://novel.naver.com/webnovel/list?novelId=1105607',
 'https://novel.naver.com/webnovel/list?novelId=1108678',
 'https://novel.naver.com/webnovel/list?novelId=1105617',
 'https://nove

In [8]:
# 모든 회차의 링크 다 모으기
total_urls = []

for title_link in title_url:
    dom = BeautifulSoup(get(title_link).text, 'html.parser')
    href = dom.select_one('.link_group a[href]').attrs['href'][:-1]    
    total_epi = int(re.search('\d+', dom.select_one('.past_number').text)[0])
    for n in range(1, total_epi+1):
        total_urls.append(base + href + str(n))

In [ ]:
len(total_urls) # 전체 18243건(11월9일 기준)

18243

In [56]:
# 내용 저장하기

for link_per_epi in total_urls: 
    full_text = ""
    dom = BeautifulSoup(get(link_per_epi).text, 'html.parser')
    plist = dom.select('.detail_view_content p')
    filename = re.search(r'\d{6,}', link_per_epi)[0] + "_" + re.sub("[\/#:?]", "", dom.select_one('title').text)

        # 레이블 유무 검사
    for tag in plist:
        if tag.select('a'):
            img_link = tag.select_one('a img').attrs['src']
            full_text += "\n" + img_link + " : " + tag.text
        else:
            full_text += "\n" + tag.text
            
    if dom.select('.talk'): # 화자 이미지가 존재한다면
        path = "C:/Users/vmfhr/OneDrive/Desktop/sw아카데미 발표/Novels/Labeled/" # 레이블 있는 파일 저장될 경로로, 알맞게 수정
        with open(f'{path}L{filename}.txt', 'w', encoding='utf-8') as fp:
            fp.write(full_text)
    else:
        path = "C:/Users/vmfhr/OneDrive/Desktop/sw아카데미 발표/Novels/Unlabeled/" # 레이블 없는 파일 저장될 경로로, 알맞게 수정
        with open(f'{path}U{filename}.txt', 'w', encoding='utf-8') as fp:
            fp.write(full_text)

KeyboardInterrupt: 

In [420]:
data = pd.DataFrame(novel_info)

In [426]:
data.to_csv('novel_info.csv', header=True, encoding='utf-8-sig', index=False)

In [ ]:
https://novel-phinf.pstatic.net/20220331_168/novel_1648717328047xCEHt_JPEG/EC9DB4ECA780EC8898_ECB59CECA285.jpg?type=w80_2
https://novel-phinf.pstatic.net/20220331_168/novel_1648717328047xCEHt_JPEG/EC9DB4ECA780EC8898_ECB59CECA285.jpg?type=w80_2
https://novel-phinf.pstatic.net/20220331_272/novel_1648717342436dUyn9_JPEG/EC9DB4EC8AB9EC98A4_ECB59CECA285.jpg?type=w80_2
https://novel-phinf.pstatic.net/20220331_25/novel_164871731157621EkR_JPEG/someone.jpg?type=w80_2

In [21]:
dom = BeautifulSoup(get("https://novel.naver.com/webnovel/detail?novelId=1051332&volumeNo=1").text, 'html.parser')
a = dom.select('.detail_view_content p')[1]

In [36]:
a

<p class="talk"><a class="talk_thum _toggleDialogLayer()" href="#"><em class="icon_mask"></em><img alt="" height="35" src="https://novel-phinf.pstatic.net/20220331_168/novel_1648717328047xCEHt_JPEG/EC9DB4ECA780EC8898_ECB59CECA285.jpg?type=w80_2" width="35"/></a>“아가씨, 저희 왔어요.”</p>

In [51]:
a.select('a img')[0].attrs['src']

'https://novel-phinf.pstatic.net/20220331_168/novel_1648717328047xCEHt_JPEG/EC9DB4ECA780EC8898_ECB59CECA285.jpg?type=w80_2'

In [48]:
dom = BeautifulSoup(get("https://novel.naver.com/webnovel/detail?novelId=1105605&volumeNo=1").text, 'html.parser')
dom.select('.detail_view_content p')[0].select('a')

[]